In [1]:
pip install ultralytics roboflow opencv-python scikit-image scikit-learn torch torchvision tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 110.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="pl67rybGD5K8ltK87WXk")
project = rf.workspace("nisha-xhe2j").project("my-first-project-s9lvk")
version = project.version(2)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to My-First-Project-2 in yolov8:: 100%|██████████| 1006/1006 [00:00<00:00, 5637.93it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # small & fast
model.train(
    data="My-First-Project-2/data.yaml",
    epochs=50,
    imgsz=640,
    project="fish_project",
    name="fish_yolo"
)


Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=My-First-Project-2/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=fish_yolo, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, po

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d12ec199a60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [13]:
import os

for root, dirs, files in os.walk("."):
    if "best.pt" in files:
        print(os.path.join(root, "best.pt"))


./fish_project/fish_yolo/weights/best.pt


In [14]:
model = YOLO("fish_project/fish_yolo/weights/best.pt")


In [15]:
import cv2
import os

video_path = "20s.mp4"
cap = cv2.VideoCapture(video_path)

os.makedirs("fish_crops", exist_ok=True)

count = 0
frame_id = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, conf=0.4)

    for r in results:
        for box in r.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            crop = frame[y1:y2, x1:x2]

            if crop.size == 0:
                continue

            crop = cv2.resize(crop, (224, 224))
            cv2.imwrite(f"fish_crops/fish_{count}.jpg", crop)
            count += 1

    frame_id += 1

cap.release()
print("Total fish cropped:", count)



0: 384x640 38 fishs, 48.7ms
Speed: 2.1ms preprocess, 48.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 39 fishs, 8.0ms
Speed: 3.2ms preprocess, 8.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 43 fishs, 8.6ms
Speed: 2.4ms preprocess, 8.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 42 fishs, 8.2ms
Speed: 2.2ms preprocess, 8.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 40 fishs, 12.0ms
Speed: 4.8ms preprocess, 12.0ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 43 fishs, 9.0ms
Speed: 2.7ms preprocess, 9.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 41 fishs, 6.2ms
Speed: 2.1ms preprocess, 6.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 40 fishs, 7.9ms
Speed: 1.9ms preprocess, 7.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 

In [16]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

effnet = models.efficientnet_b0(pretrained=True)
effnet.classifier = torch.nn.Identity()  # remove classifier
effnet = effnet.to(device)
effnet.eval()


The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 108MB/s] 


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [17]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [19]:
features = []
image_paths = []

for img_name in os.listdir("fish_crops"):
    img_path = os.path.join("fish_crops", img_name)
    img = Image.open(img_path).convert("RGB")
    img = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        feat = effnet(img).cpu().numpy().flatten()

    features.append(feat)
    image_paths.append(img_path)

features = np.array(features)
print("Feature shape:", features.shape)


Feature shape: (12375, 1280)


In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)


In [21]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(
    eps=1.2,          # important: larger to avoid over-splitting
    min_samples=10
)

labels = dbscan.fit_predict(features_scaled)
print("Total families found:", len(set(labels)) - (1 if -1 in labels else 0))


Total families found: 0


In [22]:
import numpy as np

unique, counts = np.unique(labels, return_counts=True)
print(dict(zip(unique, counts)))


{np.int64(-1): np.int64(12375)}


In [23]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(
    eps=5.0,          # VERY IMPORTANT
    min_samples=5
)

labels = dbscan.fit_predict(features_scaled)


In [24]:
unique, counts = np.unique(labels, return_counts=True)
print(dict(zip(unique, counts)))


{np.int64(-1): np.int64(11916), np.int64(0): np.int64(434), np.int64(1): np.int64(6), np.int64(2): np.int64(5), np.int64(3): np.int64(5), np.int64(4): np.int64(4), np.int64(5): np.int64(5)}


In [25]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(
    eps=1.2,          # important: larger to avoid over-splitting
    min_samples=10
)

labels = dbscan.fit_predict(features_scaled)
print("Total families found:", len(set(labels)) - (1 if -1 in labels else 0))


Total families found: 0


In [26]:
len(set(labels)) - (1 if -1 in labels else 0)


0

In [27]:
dbscan = DBSCAN(
    eps=10.0,
    min_samples=3
)


In [28]:
print(features_scaled.shape)
print(np.isnan(features_scaled).any())


(12375, 1280)
False


In [29]:
import shutil

os.makedirs("FishFamilies", exist_ok=True)

for img_path, label in zip(image_paths, labels):
    if label == -1:
        family = "Unknown"
    else:
        family = f"FishFamily_{label}"

    family_path = os.path.join("FishFamilies", family)
    os.makedirs(family_path, exist_ok=True)

    shutil.copy(img_path, family_path)


In [30]:
import shutil
shutil.make_archive("FishFamilies", "zip", "FishFamilies")


'/content/FishFamilies.zip'

In [31]:
from google.colab import files
files.download("FishFamilies.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
from ultralytics import YOLO
import cv2
import os

model = YOLO("fish_project/fish_yolo/weights/best.pt")

cap = cv2.VideoCapture("20s.mp4")

os.makedirs("tracked_fish", exist_ok=True)

saved_ids = set()   # store fish IDs already saved
fish_id_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(
        frame,
        persist=True,
        conf=0.4,
        tracker="bytetrack.yaml"
    )

    for r in results:
        if r.boxes.id is None:
            continue

        boxes = r.boxes.xyxy
        ids = r.boxes.id.cpu().numpy()

        for box, track_id in zip(boxes, ids):
            if track_id in saved_ids:
                continue   # SAME FISH → skip

            x1, y1, x2, y2 = map(int, box)
            crop = frame[y1:y2, x1:x2]

            if crop.size == 0:
                continue

            crop = cv2.resize(crop, (224, 224))
            cv2.imwrite(f"tracked_fish/fish_{track_id}.jpg", crop)

            saved_ids.add(track_id)
            fish_id_count += 1

cap.release()

print("Unique fish captured:", fish_id_count)


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 368ms
Prepared 1 package in 55ms
Installed 1 package in 9ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 1.4s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


0: 384x640 38 fishs, 9.7ms
Speed: 2.0ms preprocess, 9.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 fishs, 7.8ms
Speed: 2.2ms preprocess, 7.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 39 fishs, 5.9ms
Speed: 2.4ms preprocess, 5.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 fishs, 8.6ms
Speed: 2.0ms preprocess, 8.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 36 fishs, 7.0ms
Speed: 2.3ms preprocess, 7.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 40 fishs, 6

In [33]:
image_folder = "tracked_fish"


In [34]:
pip install hdbscan


In [35]:
import hdbscan

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=5,
    min_samples=3
)

labels = clusterer.fit_predict(features_scaled)

print("Fish families found:", len(set(labels)) - (1 if -1 in labels else 0))


invalid escape sequence '\{'


Fish families found: 3


In [36]:
import os, shutil
from collections import defaultdict

os.makedirs("FishFamilies", exist_ok=True)

family_counts = defaultdict(int)
MAX_PER_FAMILY = 30

for img_path, label in zip(image_paths, labels):
    if label == -1:
        continue

    family = f"FishFamily_{label}"

    if family_counts[family] >= MAX_PER_FAMILY:
        continue

    family_path = os.path.join("FishFamilies", family)
    os.makedirs(family_path, exist_ok=True)

    shutil.copy(img_path, family_path)
    family_counts[family] += 1


In [37]:
import shutil
shutil.make_archive("FishFamilies", "zip", "FishFamilies")


'/content/FishFamilies.zip'

In [39]:
from google.colab import files
files.download("FishFamilies.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>